# Geometrical relaxations from pretrained m3gnet model

In [1]:
import ML_library as MLL
import ase 
import matgl
import warnings

from ase.io                      import read, write
from ase.io.vasp                 import write_vasp_xdatcar
from pymatgen.io.vasp.inputs     import Poscar
from os                          import path
from __future__                  import annotations
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from pymatgen.core               import Lattice, Structure
from pymatgen.io.ase             import AseAtomsAdaptor
from matgl.ext.ase               import M3GNetCalculator, MolecularDynamics, Relaxer
from pymatgen.io.ase             import AseAtomsAdaptor

# To suppress warnings for clearer output
warnings.simplefilter('ignore')

In [2]:
# Define paths to pretrained model and structure to be relaxed

# Materials Project pretrained model as default
model_load_path = 'finetuned_model'
model_load_path = 'M3GNet-MP-2021.2.8-PES' if model_load_path is None else model_load_path
model_load_path = 'M3GNet-MP-2021.2.8-PES'

path_to_POSCAR  = 'examples'

# Structure Relaxation

To perform structure relaxation, we use the Relaxer class.

In [3]:
# Load the structure to be relaxed
atoms_ini = Structure.from_file(f'{path_to_POSCAR}/POSCAR')

In [4]:
# Load the default pre-trained model
pot = matgl.load_model(model_load_path)
relaxer = Relaxer(potential=pot)

In [5]:
# Relax the structure
relax_atoms_ini = relaxer.relax(atoms_ini, verbose=True)
atoms = relax_atoms_ini['final_structure']

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 10:38:20      -60.460709*      26.5589
FIRE:    1 10:38:20      -55.440254*      67.7939
FIRE:    2 10:38:20      -60.466652*      26.5406
FIRE:    3 10:38:20      -60.967625*       9.2130
FIRE:    4 10:38:21      -60.914856*      10.6447
FIRE:    5 10:38:21      -60.943432*       9.4187
FIRE:    6 10:38:21      -60.987305*       7.0617
FIRE:    7 10:38:21      -61.025494*       3.7695
FIRE:    8 10:38:21      -61.038826*       1.5587
FIRE:    9 10:38:21      -61.038937*       1.5364
FIRE:   10 10:38:21      -61.039150*       1.4924
FIRE:   11 10:38:21      -61.039455*       1.4284
FIRE:   12 10:38:22      -61.039860*       1.3466
FIRE:   13 10:38:22      -61.040337*       1.2502
FIRE:   14 10:38:22      -61.040894*       1.1431
FIRE:   15 10:38:22      -61.041504*       1.0301
FIRE:   16 10:38:22      -61.042255*       0.9039
FIRE:   17 10:38:22      -61.043167*       0.7702
FI

In [6]:
# Save the relaxed structure as a POSCAR file
poscar_relaxed = Poscar(atoms)
poscar_relaxed.write_file(f'{path_to_POSCAR}/CONTCAR')

In [7]:
# Or as implemented
poscar_relaxed = MLL.structural_relaxation(path_to_POSCAR, model_load_path)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 10:38:30      -60.460709*      26.5589
FIRE:    1 10:38:30      -55.440250*      67.7939
FIRE:    2 10:38:30      -60.466656*      26.5406
FIRE:    3 10:38:30      -60.967625*       9.2131
FIRE:    4 10:38:30      -60.914856*      10.6447
FIRE:    5 10:38:31      -60.943428*       9.4187
FIRE:    6 10:38:31      -60.987305*       7.0617
FIRE:    7 10:38:31      -61.025494*       3.7695
FIRE:    8 10:38:31      -61.038826*       1.5587
FIRE:    9 10:38:31      -61.038937*       1.5364
FIRE:   10 10:38:31      -61.039150*       1.4925
FIRE:   11 10:38:31      -61.039459*       1.4284
FIRE:   12 10:38:32      -61.039860*       1.3466
FIRE:   13 10:38:32      -61.040337*       1.2502
FIRE:   14 10:38:32      -61.040894*       1.1431
FIRE:   15 10:38:32      -61.041508*       1.0301
FIRE:   16 10:38:32      -61.042255*       0.9039
FIRE:   17 10:38:32      -61.043167*       0.7701
FI

# Single-shot energy calculation

Perform a single-shot calculation for final structure using M3GNetCalculator.

In [8]:
# Define the M3GNet calculator
calc = M3GNetCalculator(pot)

# Load atoms adapter and adapt structure
ase_adaptor = AseAtomsAdaptor()
adapted_atoms = ase_adaptor.get_atoms(atoms)

# Calculate potential energy
adapted_atoms.set_calculator(calc)
print(f'The calculated potential energy is {float(adapted_atoms.get_potential_energy()):.3f} eV/supercell')

The calculated potential energy is -61.342 eV


In [13]:
# Or as implemented
ssc_energy = MLL.single_shot_energy_calculations(f'{path_to_POSCAR}/CONTCAR', model_load_path)
print(f'The calculated potential energy is {ssc_energy:.3f} eV/supercell')

The calculated potential energy is -61.342 eV/supercell


In [10]:
# Try on 3x3x3 supercell
ssc_energy = MLL.single_shot_energy_calculations(f'{path_to_POSCAR}/333_supercell', model_load_path)
print(f'The calculated potential energy is {ssc_energy:.3f} eV/supercell')

The calculated potential energy is -1656.253 eV
